In [2]:
!pip install fasttext-langdetect
!pip install datasets
!pip install stopwordsiso
!pip install simplemma

  Preparing metadata (setup.py) ... done
  Created wheel for fasttext-langdetect: filename=fasttext_langdetect-1.0.5-py3-none-any.whl size=7502 sha256=6439bafc872ee84e9a5af475185e5be85de8fa93ad195a4ce4571c4ac5284d8e
  Stored in directory: /root/.cache/pip/wheels/e6/5b/5d/47e2fd5c2ff1028722739ce35f365e8f6eeb89ec97aa63e621
Successfully built fasttext-langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 MB 22.7 MB/s eta 0:00:00:00:0100:01


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
count = 0
for root, folders, filenames in os.walk('/kaggle/input'):
   print(root, folders)

/kaggle/input ['autext', 'autext2024', 'models']
/kaggle/input/autext []
/kaggle/input/autext2024 []
/kaggle/input/models []


In [4]:
from datasets import Dataset

dataset_en = Dataset.from_json("/kaggle/input/autext/train.jsonl")

dataset_en

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'text', 'label'],
    num_rows: 109663
})

In [5]:
dataset_en['label'][0:10]

['generated',
 'human',
 'generated',
 'generated',
 'generated',
 'generated',
 'generated',
 'human',
 'human',
 'human']

In [6]:
from datasets import Dataset

# Assuming you have dataset_en already loaded and containing the 'train' split

# Define a mapping from original labels to new labels
label_mapping = {"human": 0, "generated": 1}

# Function to replace labels
def map_labels(example):
    # Map the original label to the new label
    example["label"] = label_mapping[example["label"]]
    return example

# Map the labels in the 'train' split
dataset_en = dataset_en.map(map_labels)

custom_label_names = ["human", "generated"]

dataset_en['label'][0:10]

Map:   0%|          | 0/109663 [00:00<?, ? examples/s]

[1, 0, 1, 1, 1, 1, 1, 0, 0, 0]

In [7]:
from ftlangdetect import detect
import re

# Initialize an empty dictionary to store the texts for each language
language_datasets = {}
languages = ["en", "es", "ca", "pt", "eu", "gl"]

# Assuming dataset_en is a list of texts
for text,label in zip(dataset_en['text'], dataset_en['label']):
    try:
        
        language = detect(text)
    except:
        language = "Error, could not detect"

    # If the language is not already a key in the dictionary, add it with an empty list as its value
    if language['lang'] not in languages:
        continue
        if language['lang'] == 'it':
            language['lang'] = 'pt'
        elif language['lang'] == 'fr':
            language['lang'] = 'ca'
        else:
            language['lang'] = 'es'
    
    if language['lang'] not in language_datasets:
        language_datasets[language['lang']] = []

    # Append the text to the list of texts for its detected language
    language_datasets[language['lang']].append((text, label))

In [8]:
language_datasets.keys()

dict_keys(['ca', 'es', 'gl', 'en', 'pt', 'eu'])

In [9]:
# Set your threshold
threshold = 1000

# Create a new dictionary with only the items where the length of the list is greater than or equal to the threshold
#language_datasets = {language: texts for language, texts in language_datasets.items() if len(texts) >= threshold}

# Assuming language_datasets is your dictionary of languages and texts
for language, texts in language_datasets.items():
    print(f"Language: {language}, Number of texts: {len(texts)}")

Language: ca, Number of texts: 16413
Language: es, Number of texts: 22353
Language: gl, Number of texts: 11448
Language: en, Number of texts: 25769
Language: pt, Number of texts: 20034
Language: eu, Number of texts: 13574


In [10]:
import pandas as pd

# Dictionary to store DataFrames
dataframes = {}

# Convert each list of tuples to DataFrame
for lang, data in language_datasets.items():
    df = pd.DataFrame(data, columns=['text', 'label'])
    dataframes[lang] = df

# Accessing DataFrames
print(dataframes['en'])
print(dataframes['eu'])

                                                    text  label
0      Another magnificent chapter from @nbchannibal ...      0
1      Paul simon is a great artist with multiple hit...      0
2      R hoarsely, "don"t be frightened. father"s tum...      0
3      Sri lanka is set to return 21 containers of wa...      1
4      Milo lompar (; born 19 april 1962) is a serbia...      0
...                                                  ...    ...
25764  Rtion for him. every little while, however, on...      0
25765  Mark xvii or mark 17 often refers to the 17th ...      0
25766  Marianne, once recovered from her injury, was ...      1
25767  The irish president has signed the "long-await...      1
25768  Sure! wikipedia articles should be well-writte...      1

[25769 rows x 2 columns]
                                                    text  label
0      """basque artikulu hau: lasioglossum figueresi...      1
1      Ez errepikatu gonbita edo emandako esaldiak. e...      1
2      Hori de

In [11]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
stopwords_en = stopwords.words("english")
stopwords_es = stopwords.words("spanish")
stopwords_pt = stopwords.words("portuguese")

import stopwordsiso as stopwordsd

stopwords_eu = stopwordsd.stopwords("eu")
stopwords_gl = stopwordsd.stopwords("gl")
stopwords_ca = stopwordsd.stopwords("ca")

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import EnglishStemmer, PortugueseStemmer, SpanishStemmer
import re

from simplemma import simple_tokenizer
from simplemma import text_lemmatizer

def clean_text_en(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=word_tokenize(text, language="english")
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwords.words("english")]
    # usar los stems
    tokens = [PorterStemmer().stem(word) for word in tokens]
    # eliminamos las palabras con menos de 3 caráceres
    min_length = 3
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    p = re.compile('^[a-zA-Z]+$');

    filtered_tokens=[]
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens.append(token)

    return filtered_tokens

def clean_text_es(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=word_tokenize(text, language="spanish")
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwords.words("spanish")]
    # usar los stems
    tokens = [SpanishStemmer().stem(word) for word in tokens]
    # eliminamos las palabras con menos de 3 caráceres
    min_length = 3
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    p = re.compile('^[a-zA-Z]+$');

    filtered_tokens=[]
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens.append(token)

    return filtered_tokens

def clean_text_pt(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=word_tokenize(text, language="portuguese")
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwords.words("portuguese")]
    # usar los stems
    tokens = [PortugueseStemmer().stem(word) for word in tokens]
    # eliminamos las palabras con menos de 3 caráceres
    min_length = 3
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    p = re.compile('^[a-zA-Z]+$');

    filtered_tokens=[]
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens.append(token)

    return filtered_tokens

def clean_text_gl(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=simple_tokenizer(text)
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwordsd.stopwords("gl")]
    # usar los stems
    tokens = [text_lemmatizer(word, lang='gl') for word in tokens]
    # eliminamos las palabras con menos de 3 caráceres
    min_length = 3
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    p = re.compile('^[a-zA-Z]+$');

    filtered_tokens=[]
    for token in tokens:
        if len(token[0])>=min_length and p.match(token[0]):
            filtered_tokens.append(token[0])

    return filtered_tokens

def clean_text_eu(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=simple_tokenizer(text)
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwordsd.stopwords("eu")]
    # usar los stems
    tokens = [text_lemmatizer(word, lang='eu') for word in tokens]
    # eliminamos las palabras con menos de 3 caráceres
    min_length = 3
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    p = re.compile('^[a-zA-Z]+$');

    filtered_tokens=[]
    for token in tokens:
        if len(token[0])>=min_length and p.match(token[0]):
            filtered_tokens.append(token[0])

    return filtered_tokens

def clean_text_ca(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=simple_tokenizer(text)
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwordsd.stopwords("ca")]
    # usar los stems
    tokens = [text_lemmatizer(word, lang='ca') for word in tokens]
    # eliminamos las palabras con menos de 3 caráceres
    min_length = 3
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    p = re.compile('^[a-zA-Z]+$');

    filtered_tokens=[]
    for token in tokens:
        if len(token[0])>=min_length and p.match(token[0]):
            filtered_tokens.append(token[0])

    return filtered_tokens

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
clean_text_eu(dataframes['eu']['text'][0])

['basque',
 'artikulu',
 'lasioglossum',
 'figueresi',
 'lasioglossum',
 'buruzko',
 'lekuz',
 'aldetik',
 'mahaigaindiko',
 'blokean',
 'iparraldeko',
 'lurralde',
 'lurraldeetan',
 'lehia',
 'lardo',
 'zizkaidoa',
 'eman',
 'lasioglossum',
 'megachilenoak',
 'gainako',
 'ibaiako']

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import joblib
from sklearn.metrics import classification_report, f1_score

languages = ["en", "es", "ca", "pt", "eu", "gl"]

clean_text_functions = {
    "en": clean_text_en,
    "es": clean_text_es,
    "ca": clean_text_ca,
    "pt": clean_text_pt,
    "eu": clean_text_eu,
    "gl": clean_text_gl,
}

for lang, dataset in dataframes.items():
    print(f"\nlanguage: {lang}\n")
    # train test splits
    if lang in ["ca", "es", "gl"]:#, "pt", "en"]:
        continue
        
    #train_df, test_df = train_test_split(dataset, test_size=0.2, random_state=42)
    train_df = dataset
    print("Train set shape:", train_df.shape)
    #print("Test set shape:", test_df.shape)
    
    # Transformar labels a numéricas
    y_train = train_df['label'].tolist()
    
    #y_test = test_df['label'].tolist()
    
    # Pipeline con SVC 
    pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text_functions[lang])),
    ('tf', TfidfTransformer()),
    ('svm', SVC()),
    ])

    X_train = train_df['text'].tolist()

    # entrenamos el pipeline
    pipeline.fit(X_train, y_train)

    # Guardamos el modelo
    joblib.dump(pipeline, f'full_model_{lang}.pkl')
      
    """
    # Predicciones sobre test y resultados
    X_test= test_df['text'].tolist()
    predictions = pipeline.predict(X_test)
    print(classification_report(y_test, predictions, target_names=LABELS))
    print(f"F1 Score: {f1_score(y_test, predictions, average='macro')}")
    """
    

2024-05-18 14:01:55.456922: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 14:01:55.457078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 14:01:55.613650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



language: ca


language: es


language: gl


language: en

Train set shape: (25769, 2)

language: pt

Train set shape: (20034, 2)

language: eu

Train set shape: (13574, 2)


In [15]:
from datasets import Dataset

dataset_test = Dataset.from_json("/kaggle/input/autext/test.jsonl")

dataset_test

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'text'],
    num_rows: 43365
})

In [16]:
from ftlangdetect import detect
import re

# Initialize an empty dictionary to store the texts for each language
language_datasets = {}
languages = ["en", "es", "ca", "pt", "eu", "gl"]

# Assuming dataset_en is a list of texts
for id_t,text in zip(dataset_test['id'], dataset_test['text']):
    try:
        language = detect(text)
    except:
        language = "Error, could not detect"

    # If the language is not already a key in the dictionary, add it with an empty list as its value
    if language['lang'] not in languages:
        if language['lang'] == 'it':
            language['lang'] = 'pt'
        elif language['lang'] == 'fr':
            language['lang'] = 'ca'
        else:
            language['lang'] = 'es'

    if language['lang'] not in language_datasets:
        language_datasets[language['lang']] = []

    # Append the text to the list of texts for its detected language
    language_datasets[language['lang']].append((id_t,text))

In [17]:
# Set your threshold
threshold = 1000

# Create a new dictionary with only the items where the length of the list is greater than or equal to the threshold
#language_datasets = {language: texts for language, texts in language_datasets.items() if len(texts) >= threshold}

# Assuming language_datasets is your dictionary of languages and texts
for language, texts in language_datasets.items():
    print(f"Language: {language}, Number of texts: {len(texts)}")

Language: en, Number of texts: 10324
Language: ca, Number of texts: 8013
Language: pt, Number of texts: 9003
Language: es, Number of texts: 9116
Language: eu, Number of texts: 5518
Language: gl, Number of texts: 1391


In [18]:
import pandas as pd

# Dictionary to store DataFrames
dataframes = {}

label_mapping = {"human": 0, "generated": 1}

def map_labels(example):
        # Map the original label to the new label
        example["label"] = label_mapping[example["label"]]
        return example

# Convert each list of tuples to DataFrame
for lang, data in language_datasets.items():
    df = pd.DataFrame(data, columns=['id', 'text'])

    dataframes[lang] = Dataset.from_pandas(df)

dataframes

{'en': Dataset({
     features: ['id', 'text'],
     num_rows: 10324
 }),
 'ca': Dataset({
     features: ['id', 'text'],
     num_rows: 8013
 }),
 'pt': Dataset({
     features: ['id', 'text'],
     num_rows: 9003
 }),
 'es': Dataset({
     features: ['id', 'text'],
     num_rows: 9116
 }),
 'eu': Dataset({
     features: ['id', 'text'],
     num_rows: 5518
 }),
 'gl': Dataset({
     features: ['id', 'text'],
     num_rows: 1391
 })}

In [19]:
import joblib

languages = ["en", "es", "ca", "pt", "eu", "gl"]

models_svm_lang = {}

for lang in languages:

    models_svm_lang[lang] = joblib.load(f"/kaggle/input/models/full_model_{lang}.pkl")

models_svm_lang

{'en': Pipeline(steps=[('bow',
                  CountVectorizer(analyzer=<function clean_text_en at 0x7d153f862dd0>)),
                 ('tf', TfidfTransformer()), ('svm', SVC())]),
 'es': Pipeline(steps=[('bow',
                  CountVectorizer(analyzer=<function clean_text_es at 0x7d14b80f6830>)),
                 ('tf', TfidfTransformer()), ('svm', SVC())]),
 'ca': Pipeline(steps=[('bow',
                  CountVectorizer(analyzer=<function clean_text_ca at 0x7d14b80f6a70>)),
                 ('tf', TfidfTransformer()), ('svm', SVC())]),
 'pt': Pipeline(steps=[('bow',
                  CountVectorizer(analyzer=<function clean_text_pt at 0x7d151aa5e8c0>)),
                 ('tf', TfidfTransformer()), ('svm', SVC())]),
 'eu': Pipeline(steps=[('bow',
                  CountVectorizer(analyzer=<function clean_text_eu at 0x7d151aa5e9e0>)),
                 ('tf', TfidfTransformer()), ('svm', SVC())]),
 'gl': Pipeline(steps=[('bow',
                  CountVectorizer(analyzer=<function c

In [20]:
import pickle

prefix = 'svm'

for lang in languages:

    print(lang)
    X_test = dataframes[lang]['text']
    X_ids = dataframes[lang]['id']

    y_preds = models_svm_lang[lang].predict(X_test)

    svm_preds = [(id_t, pred) for id_t,pred in zip(X_ids, y_preds)]

    with open(f"/kaggle/working/{prefix}_test_{lang}.pkl", "wb") as f:
        pickle.dump(svm_preds, f)



svm_preds[0:10]

en
es
ca
pt
eu
gl


[('151315', 1),
 ('25491', 1),
 ('91451', 1),
 ('88976', 1),
 ('105491', 0),
 ('80721', 1),
 ('109998', 1),
 ('93086', 1),
 ('150366', 1),
 ('7212', 1)]

In [74]:
from sklearn.model_selection import train_test_split

dataframe_en = dataframes['en']  
dataframe_en = dataframes['es']  
dataframe_en = dataframes['gl']  

# Perform train-test split
train_df_en, test_df_en = train_test_split(dataframe_en, test_size=0.2, random_state=42)

# Output the shapes of train and test sets
print("Train set shape:", train_df_en.shape)
print("Test set shape:", test_df_en.shape)
train_df_en

Train set shape: (9158, 2)
Test set shape: (2290, 2)


,text,label
2018,"Vilela, lamas, san sadurniño é un concello sit...",generated
9654,Non podo evitar sentir unha certa decepción po...,generated
11361,Con esta iniciativa queriamos convidar os rapa...,human
7800,"A fonsagrada, 1951) tiña claro que a súa vocac...",generated
3353,Delle é un lugar da parroquia de ferreiros no ...,human
...,...,...
11284,"Pola súa parte, nerea lage e marcos burgo eran...",human
5191,"Prudencia santasmarinas raposo, nada en 1943 e...",human
5390,"O escorial, recaré, o valadouro é unha localid...",generated
860,"A navalla de occam, ou principio da parsimonia...",human


In [75]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

y_train = train_df_en['label'].tolist()
y_test = test_df_en['label'].tolist()

le = LabelEncoder()

print("antes de transform: ", y_train[:10])
y_train = le.fit_transform(y_train)
LABELS = le.classes_
NUM_LABELS = len(LABELS)
print("después de transform:", y_train[:10])

print("antes de transform: ", y_test[:10])
y_test = le.transform(y_test)
print("después de transform:", y_test[:10])

idx2label={}
label2idx={}
for index, label in enumerate(LABELS):
    label2idx.update([(label, index)])
    idx2label.update([(index, label)])
#y_train_cat = to_categorical(y_train)
#y_test_cat = to_categorical(y_test)
print('Labels:', label2idx)

antes de transform:  ['generated', 'generated', 'human', 'generated', 'human', 'human', 'human', 'human', 'human', 'generated']
después de transform: [0 0 1 0 1 1 1 1 1 0]
antes de transform:  ['human', 'human', 'generated', 'human', 'generated', 'human', 'human', 'human', 'human', 'human']
después de transform: [1 1 0 1 0 1 1 1 1 1]
Labels: {'generated': 0, 'human': 1}


In [76]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


# definimos un pipeline que primero transforma los textos a BoW
# después los textos se transforma en formato tfidf
# finalmente, se aplica el algoritmo
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('svm', SVC()),
])

# totamos los textos
X_train = train_df_en['text'].tolist()

# entrenamos el pipeline
pipeline.fit(X_train, y_train)


Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7fe0f07eecb0>)),
                ('tf', TfidfTransformer()), ('svm', SVC())])

In [79]:
import joblib
joblib.dump(pipeline, 'model_gl.pkl')

['model_gl.pkl']

In [77]:
# totamos los textos del conjunto test
X_test= test_df_en['text'].tolist()
# las labels ya fueron cargadas y transformadas en el apartado de Label encoding.

# usamos el modelo para inferir las predicciones
predictions = pipeline.predict(X_test)


In [78]:
from sklearn.metrics import classification_report
print( classification_report(y_test, predictions, target_names=LABELS))


              precision    recall  f1-score   support

   generated       0.87      0.86      0.87      1234
       human       0.84      0.86      0.85      1056

    accuracy                           0.86      2290
   macro avg       0.86      0.86      0.86      2290
weighted avg       0.86      0.86      0.86      2290



In [72]:
X_test= test_df_en['text'].tolist()
clean_text(X_test[0]), X_test[0]

hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey


(['oviedo',
  'rugby',
  'club',
  'entidade',
  'deportivo',
  'cidade',
  'oviedo',
  'dedicar',
  'rugby',
  'actualmente',
  'militar',
  'honrar',
  'segundo',
  'rugby',
  'historiar',
  'oviedo',
  'fundar',
  'ano',
  'comer',
  'resultar',
  'dous',
  'club',
  'vello',
  'cau',
  'oviedo',
  'economicas',
  'dous',
  'club',
  'habitual',
  'primeiro',
  'nacional',
  'daquela',
  'segundo',
  'rugby',
  'nacional',
  'tempada',
  'novo',
  'equipar',
  'ascender',
  'primeiro',
  'nacional',
  'acadar',
  'caber',
  'salientar',
  'dende',
  'ano',
  'atar',
  'actualidade',
  'oviedo',
  'levar',
  'militar',
  'sempre',
  'nalgunha',
  'rugby',
  'seguinte',
  'tornear',
  'equipar',
  'rematar',
  'cal',
  'ascender',
  'honrar',
  'debutar',
  'tempada',
  'nesa',
  'primeiro',
  'elite',
  'rugby',
  'oviedo',
  'rematar',
  'cal',
  'descender',
  'segundo',
  'metade',
  'oviedo',
  'rematar',
  'primeiro',
  'tres',
  'vez',
  'grupo',
  'primeiro',
  'nacional',
  '

In [ ]:
import fasttext.util
fasttext.util.download_model('gl', if_exists='ignore')
ft = fasttext.load_model('cc.gl.300.bin')